In [1]:
import string
import re
import operator

# table renvoyant une liste de textes (sous forme d'ints représentant le numéro des textes) pour chaque mot
tableMotsGenerale = {}
# pour chaque numéro de texte renvoie le hashSet représentant les valeurs TFIDF de chaque mot
tableTextes = {}


def acquisition(path):
    
    f=open(path,'r')
    
    # masque pour détecter les débuts de textes
    regexp=re.compile("^\.I\s*([0-9]*)\s*")
    
    # numéro du texte courant, valeur abérante par défaut
    numero = -1
    
    # tableau contenant chaque ligne du fichier
    listeLignes = f.readlines()
    
    # le nombre de mots rencontrés
    nombreMots = 0
    
    
    for ligne in listeLignes:
        # supprime les espaces et sauts de lignes en début et fin de chaîne
        ligne.lstrip
        ligne.rstrip("\n")
        
        # on cherche la marque du début d'un nouveau ou du premier texte
        result = regexp.search(ligne)
        if result != None:
            # teste si on a déjà rencontré un texte avant
            if numero != -1:
                for mot,valeur in tableTextes[numero].items():
                    # TF, sans IDF car il manque le nombre de textes dans lesquels le mot apparait
                    tableTextes[numero][mot]=(valeur/nombreMots)
            # on capture son index 
            numero = int(result.group(1))
            # on initialise sa table de mots
            tableTextes[numero] = {}
             # on initialise le nombre de mots de celui-ci
            nombreMots = 0
            
        else:
            
            # on casse la ligne du document
            mots = ligne.split()
            
            for mot in mots:
                nombreMots+=1
                # on supprime la ponctuation éventuelle
                mot = re.sub("[\.,;:\?!]","",mot)
                # si le mot existe déjà dans la table on augmente son nombre d'apparitions
                if mot in tableTextes[numero]:
                    tableTextes[numero][mot]+=1
                # dans le cas contraire on ajoute le mot dans la table
                else:
                    tableTextes[numero][mot]=1
                    # si le mot a déjà été vu pour un autre texte, on ajoute ce texte dans la table générale
                    if mot in tableMotsGenerale:
                        tableMotsGenerale[mot].append(numero)
                    # sinon on ajoute le mot dans la table générale et on y associe ce texte
                    else:
                        tableMotsGenerale[mot]=[numero]

    # on a parcouru tous les termes, on peut appliquer l'IDF du TFIDF au TF
    for texte,mapMots in tableTextes.items():
        for mot,tf in tableTextes[texte].items():
            tableTextes[texte][mot]*=math.log(len(tableTextes)/len(tableMotsGenerale[mot]))
    
    
    f.close()
    
    return

# textes : acquisition met à jour la base de données des corpus
acquisition("./CISI.ALLnettoye")

NameError: name 'math' is not defined

In [ ]:
def traitementRequete(Requete):
    # Donne, pour chaque texte, un score de pertinance par rapport à la requête
    scores = {}
    
    # Calcul des scores
    mots = Requete.split()
    # trouve le score max
    max = 0
    for mot in mots:
        # on supprime la ponctuation éventuelle
        mot = re.sub("[\.,;:\?!]","",mot)
        if mot in tableMotsGenerale:
            listeTextes = tableMotsGenerale[mot]
            for texte in listeTextes:
                if texte in scores:
                    scores[texte]+=tableTextes[texte][mot]
                else:
                    scores[texte]=tableTextes[texte][mot]
                if scores[texte] > max:
                    max = scores[texte]

                
    
    # enlève les textes avec un petit score (inférieur à la moitié du meilleur)
    copie = dict(scores)
    for texte,score in scores.items():
        if score<(max/2):
            del copie[texte]
    scores = copie
    
    return scores


def printScores(scores):
    print((sorted(scores.items(), key=operator.itemgetter(1)))[::-1])

scores = traitementRequete("Dewey")
printScores(scores)



In [ ]:
def acquisitionDesRequetes(path):
    f=open(path,'r')
    
    # si true, on capture une requête
    catch = False
    
    # Table contenant pour chaque numéro de requête un texte de requête
    tableRequete={}
    
    # masque pour détecter les numéros de requêtes
    regexNumero=re.compile("^\.I\s*([0-9]*)\s*")
    
    # masque pour détecter les débuts de requêtes
    regexContenu=re.compile("^\.W")
    
    # masque pour détecter les fins de requêtes
    regexFinCatch=re.compile("^\.[BI]")
    
    # numéro de la requête courante
    numero = 1
    
    # tableau contenant chaque ligne du fichier
    listeLignes = f.readlines()
    
    for ligne in listeLignes:
        # supprime les espaces et sauts de lignes en début et fin de chaîne
        ligne.lstrip
        ligne.rstrip("\n")
        
        # fin de texte
        result = regexFinCatch.search(ligne)
        if result != None:
            catch = False
        
        if catch:
            tableRequete[numero]+=ligne
        
        # numero de requête
        result = regexNumero.search(ligne)
        if result != None:
            # on capture son index 
            numero = int(result.group(1))
            # on initialise son contenu
            tableRequete[numero] = ""
        # debut de texte
        result = regexContenu.search(ligne)
        if result != None:
            catch = True
            
    f.close()
    
    return tableRequete



In [ ]:
def evaluationListeRequetes(listeRequetes):
    listeScoresRequetes = {}
    for numero,requete in listeRequetes.items():
        listeScoresRequetes[numero] = traitementRequete(requete)
    #print(listeScoresRequetes[8])    
    return listeScoresRequetes
        

def ecrireListeScoresRequetes(path,listeScoresRequetes):
    f=open(path,'w')
    for numeroRequete,scores in listeScoresRequetes.items():
        for numeroTexte,score in scores.items():
            f.write(str(numeroRequete)+" "+str(numeroTexte)+" "+str(score)+'\n')
    
    
    f.close()
    return

# textes : acquisition met à jour la base de données des corpus
acquisition("./CISI.ALLnettoye")
# requêtes : ecrireListeScoresRequetes écrit dans un fichier le score de pertinence pour chaque texte obtenu 
# pour chaque requête (evaluationlisteRequetes apelle traitementRequete sur chaque requête venant de
# acquisitionDesRequetes)
ecrireListeScoresRequetes("./resultats",evaluationListeRequetes(acquisitionDesRequetes("./CISI_dev.QRY")))

# Evaluer les résultats avec le script perl : ./eval.pl CISI_dev.REL resultats 